In [1]:
from citipy import citipy
import numpy as np
import pandas as pd

In [2]:
# create a set of random latittude and longtiudes combinations

lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

In [3]:
#Add the latitudes and longtitudes to a list
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

620

In [5]:
# Import the requests library.
import requests
import time
from datetime import datetime
# Import the API key.
#from config import weather_api_key
weather_api_key = "704b939c1f54b48486e1b5a5c91ec916"

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=704b939c1f54b48486e1b5a5c91ec916


In [7]:
city_data = []
print("Beginning Data Retrieval")
print("-" * 25)

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        try:
            city_rain_inches = city_weather["rain"]["1h"]
        except:
            city_rain_inches = 0
            
        try:
            city_snow_inches = city_weather["rain"]["1h"]
        except:
            city_snow_inches = 0
            
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          #"Current Temperature": city_weather_description,
                          #"Rain (inches)": city_rain_inches,
                          #"Snow (inches)": city_snow_inches,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Temperature": city_weather_description})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval
-------------------------
Processing Record 1 of Set 1 | pisco
Processing Record 2 of Set 1 | yellowknife
Processing Record 3 of Set 1 | ponta delgada
Processing Record 4 of Set 1 | busselton
Processing Record 5 of Set 1 | barrow
Processing Record 6 of Set 1 | ribeira grande
Processing Record 7 of Set 1 | xining
City not found. Skipping...
Processing Record 8 of Set 1 | tautira
City not found. Skipping...
Processing Record 9 of Set 1 | berlevag
City not found. Skipping...
Processing Record 10 of Set 1 | nouadhibou
City not found. Skipping...
Processing Record 11 of Set 1 | jamestown
City not found. Skipping...
Processing Record 12 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 13 of Set 1 | kapaa
City not found. Skipping...
Processing Record 14 of Set 1 | soyo
Processing Record 15 of Set 1 | evensk
Processing Record 16 of Set 1 | rikitea
Processing Record 17 of Set 1 | atherton
Processing Record 18 of Set 1 | hilo
Processing Record 19 of Set 1

In [9]:
#convert list of dictionaries to a pandara DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Temperature
0,Pisco,-13.7000,-76.2167,69.85,78,0,8.05,clear sky
1,Yellowknife,62.4560,-114.3525,-13.47,98,33,5.66,scattered clouds
2,Ponta Delgada,37.7333,-25.6667,57.92,72,40,10.36,scattered clouds
3,Busselton,-33.6500,115.3333,86.94,32,1,8.75,clear sky
4,Barrow,71.2906,-156.7887,-12.98,70,0,8.05,clear sky
5,Ribeira Grande,38.5167,-28.7000,58.32,59,100,18.66,overcast clouds
6,Soyo,-6.1349,12.3689,80.15,84,100,3.69,overcast clouds
7,Evensk,61.9500,159.2333,25.86,59,37,18.28,scattered clouds
8,Rikitea,-23.1203,-134.9692,78.75,70,59,5.55,broken clouds
9,Atherton,53.5237,-2.4935,53.29,81,100,1.99,overcast clouds


In [13]:
new_column_order = sorted(city_data_df.columns.tolist())
city_data_df = city_data_df[new_column_order]

# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")